# CDalgs - Community detection algorithms

There is an overview of four algorithms for community dentection on undirected graphs. 
- ModMax - modularity maximisation of the whole graph. 
- Louvain method - hierarchically based modularity maximisation. 
- CDEP algorithm - the third one performs graph compression and seeding expansion. 
- NCClustering - greedy approach of near clique (dense subgraphs) clustering.


Load libraries

In [ ]:
using CDalgs
using MLDatasets
using LinearAlgebra, StatsBase
using Graphs, SimpleWeightedGraphs
using Plots

Load dataset & define clustering and graph methods

In [ ]:
dataset = MNIST(:train).features
#clustering = g -> modmax_clustering(g) 
#clustering = g -> louvain_clustering(g) 
#clustering = g -> cdep_clustering(g) 
clustering = g -> nc_clustering(g)
create_graph = d -> correlation_graph(d)

Prepare data and correlation graph

In [ ]:
n= size(dataset)[1]

data_vec = collect(hcat(
    map(x -> vec(x), eachslice(dataset, dims=3))...
)')

@show size(data_vec)
    
g = create_graph(data_vec)

Perform community detection (graph clustering)

In [ ]:
clusters = clustering(g)
clusters_matrix = transpose(reshape(clusters, (n, n)))

Create color map

In [ ]:
clusters_matrix_colors = fill(RGB(0, 0, 0), (n, n))
colors_vec = map(x->RGB(rand(1)[1], rand(1)[1], rand(1)[1]), 1:length(unique(clusters)))
for i::Int64=1:n for j::Int64=1:n
    clusters_matrix_colors[i, j] = colors_vec[clusters_matrix[i, j]]
end end
plot(clusters_matrix_colors)

Display clustering of different methods

In [ ]:
function clusterdataset(dataset::T, clustering::Function,
    graph::Function) where T<:AbstractArray

    n::Int64 = size(dataset)[1]

    data_vec::Matrix = collect(hcat(
    map(x -> vec(x), eachslice(dataset, dims=3))...
    )')

    g::SimpleWeightedGraph = graph(data_vec)

    clusters::Vector{Int64} = clustering(g)
    clusters_matrix::Matrix{Int64} = transpose(reshape(clusters, (n, n)))

    clusters_matrix_colors = fill(RGB(0, 0, 0), (n, n))
    colors_vec = map(x->RGB(rand(1)[1], rand(1)[1], rand(1)[1]), 1:length(unique(clusters)))
    for i::Int64=1:n for j::Int64=1:n
    clusters_matrix_colors[i, j] = colors_vec[clusters_matrix[i, j]]
    end end
    clusters_matrix, plot(clusters_matrix_colors, title=String(Symbol(clustering)))
end

dataset = MNIST(:train).features

imgs = []
#append!(imgs, [clusterdataset(dataset, modmax_clustering, d->correlation_graph(d))[2]])
append!(imgs, [clusterdataset(dataset, louvain_clustering, d->correlation_graph(d))[2]])
#append!(imgs, [clusterdataset(dataset, cdep_clustering, d->correlation_graph(d))[2]])
append!(imgs, [clusterdataset(dataset, nc_clustering, d->correlation_graph(d))[2]])

plot(imgs..., layout=(1, 2))